API Practice

In [6]:
# Import the necessary libraries
import requests   # to call the API
import pandas as pd  # to organize data into a table
from datetime import datetime  # to add timestamps

# Define the CoinGecko API endpoint
url = "https://api.coingecko.com/api/v3/simple/price"

# Set the parameters (which coins, and which currency)
params = {
    "ids": "bitcoin,ethereum,dogecoin",   # coins we want
    "vs_currencies": "usd"                # convert to USD
}

# Send a GET request to the API
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # parse the JSON response
else:
    data = {"error": f"Failed to fetch prices. Status: {response.status_code}"}

# If no error, process the data
if "error" not in data:
    # Convert JSON into a DataFrame for nice table format
    df = pd.DataFrame(data).T  # transpose so coins are rows
    df["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # add timestamp
    display(df)  # show table in Jupyter
else:
    print(data["error"])

,usd,timestamp
bitcoin,115130.000000,2025-08-18 18:15:49
dogecoin,0.222497,2025-08-18 18:15:49
ethereum,4287.620000,2025-08-18 18:15:49


In [3]:
import requests

symbol = "AC"  # default ticker
url = f"https://spring-psx-api.herokuapp.com/stocks/{symbol}"
resp = requests.get(url)
data = resp.json() if resp.status_code == 200 else {"error": "Failed to fetch data"}
data

{'error': 'Failed to fetch data'}

In [7]:
pip install dash dash-bootstrap-components requests

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.9 MB ? eta -:--:--
   ----------- ---------------------------- 2.4/7.9 MB 10.0 MB/s eta 0:00:01
   ------------------------- -------------- 5.0/7.9 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------  7.9/7.9 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 7.9/7.9 MB 11.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   ----------- ---------------------------- 2.9/9.8 MB 14.2 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.8 MB 14.1 MB/s eta 0:00:01
   ------------------------------------ --- 8.9/9.8 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 13.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import requests
import dash
from dash import html, dcc
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc
import datetime

# ----------------------
# Setup Dash App
# ----------------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])  # dark theme

# ----------------------
# Define layout
# ----------------------
app.layout = dbc.Container([
    html.H1("🚀 Live Crypto Prices Dashboard", className="text-center text-light mb-4"),

    # Display last updated time
    html.Div(id="last-update", className="text-info mb-3"),

    # Placeholders for live prices
    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardHeader("Bitcoin (BTC)"),
            dbc.CardBody(html.H2(id="btc-price", className="text-success"))
        ], className="shadow-lg rounded-3"), width=4),

        dbc.Col(dbc.Card([
            dbc.CardHeader("Ethereum (ETH)"),
            dbc.CardBody(html.H2(id="eth-price", className="text-success"))
        ], className="shadow-lg rounded-3"), width=4),

        dbc.Col(dbc.Card([
            dbc.CardHeader("Dogecoin (DOGE)"),
            dbc.CardBody(html.H2(id="doge-price", className="text-success"))
        ], className="shadow-lg rounded-3"), width=4),
    ], className="mb-4"),

    # Auto refresh every 10 seconds
    dcc.Interval(
        id="interval-component",
        interval=10*1000,  # 10 seconds
        n_intervals=0
    )
], fluid=True)

# ----------------------
# Define callback (updates prices every interval)
# ----------------------
@app.callback(
    [Output("btc-price", "children"),
     Output("eth-price", "children"),
     Output("doge-price", "children"),
     Output("last-update", "children")],
    [Input("interval-component", "n_intervals")]
)
def update_prices(n):
    # API endpoint
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {"ids": "bitcoin,ethereum,dogecoin", "vs_currencies": "usd"}

    # Fetch data
    try:
        response = requests.get(url, params=params)
        data = response.json()
        btc = f"${data['bitcoin']['usd']:,}"
        eth = f"${data['ethereum']['usd']:,}"
        doge = f"${data['dogecoin']['usd']:,}"
        timestamp = f"Last updated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    except:
        btc, eth, doge, timestamp = "Error", "Error", "Error", "API Fetch Failed"

    return btc, eth, doge, timestamp

# ----------------------
# Run app
# ----------------------
app.run(debug=True)